In [1]:
import pandas as pd
import os
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
import pickle
import numpy as np
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import word_tokenize

lemmatizer = WordNetLemmatizer()
ps = PorterStemmer()

C:\Users\DELL\AppData\Local\Temp\ipykernel_11416\1027705287.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
corpus_path = "F:/IR Project/symtoms_redormulation/corpus"
files = os.listdir(corpus_path)
dis_symp = {}
for i in files:
    with open(corpus_path+"/"+i,'r',encoding='utf-8') as f:
        data = f.read().split("\n")
        for line in data:
            if line.split(':')[0] == 'Symptoms':
                dis_symp[i[:-4]] = re.sub(r'\([^)]*\)', '', re.sub(r'\[[^\]]*\]', '', re.sub(r'\d', '', line.split(":")[1]))).split(",")
    f.close()

In [4]:
wnl = WordNetLemmatizer()

def penn2morphy(penntag):
    morphy_tag = {'NN': 'n', 'JJ': 'a', 'VB': 'v', 'RB': 'r'}
    return morphy_tag.get(penntag[:2], 'n')

def lemmatize_sent(text):
    tokens = word_tokenize(text)
    pos_tags = nltk.pos_tag(tokens)
    lemmatized_words = [wnl.lemmatize(word.lower(), pos=penn2morphy(tag)) for word, tag in pos_tags]
    return lemmatized_words[0]

In [5]:
all_symptoms = set()
for i in dis_symp:
    for j in dis_symp[i]:
        word = j.strip().lower()
        if word=="":
            dis_symp[i].remove('')
            continue
        if word=='none':
            dis_symp[i].remove(j)
            continue
        if len(word.split(' '))==1:
            all_symptoms.add(lemmatize_sent(word))
        else:
            all_symptoms.add(word)
all_symptoms = list(all_symptoms)

In [6]:
def remove_stopwords_from_file(content):
    stop_words = set(stopwords.words('english'))
    tokens = nltk.word_tokenize(content)
    filtered_tokens = [word.lower() for word in tokens if word.lower() not in stop_words]
    return filtered_tokens

In [7]:
symps = {}
for i in all_symptoms:
    symps[i] = remove_stopwords_from_file(i)

for i in symps:
    temp = []
    for j in symps[i]:
        temp.append(lemmatize_sent(j))
    symps[i] = temp

In [8]:
symps = {key: ' '.join(words) for key, words in symps.items()}

In [9]:
docs = [doc for _,doc in symps.items()]
headings = [heading for heading,_ in symps.items()]

In [10]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf_vectorizer = TfidfVectorizer(use_idf=True)
tfidf_matrix = tfidf_vectorizer.fit_transform(docs)
with open('./Dataset/tfidf_vectorizer.pkl', 'wb') as handle:
    pickle.dump(tfidf_vectorizer, handle)
df = pd.DataFrame(tfidf_matrix.toarray(), columns=tfidf_vectorizer.get_feature_names_out())
df.index = headings
df.to_pickle('./Dataset/tf_idf_matrix.pkl')

In [11]:
dis_symp1 = {}
vocab = set()
for i in dis_symp:
    temp = []
    for j in dis_symp[i]:
        word = j.strip().lower()
        if word=="":
            continue
        elif word=='none':
            continue
        elif len(word.split(' '))==1:
            temp.append(lemmatize_sent(word))
            vocab.add(lemmatize_sent(word))
        else:
            temp.append(word)
            vocab.add(word)
    dis_symp1[i] = temp

In [12]:
inverted_index = {}
for i in vocab:
    inverted_index[i] = []

In [13]:
for i in dis_symp1:
    for j in vocab:
        if j in dis_symp1[i]:
            inverted_index[j].append(i)
with open('./Dataset/inverted_index.pkl','wb') as i_file:
    pickle.dump(inverted_index,i_file)

In [14]:
dis_symp1

{'Abscess': ['redness', 'pain', 'swell'],
 'Acquired Capillary Haemangioma of Eyelid': ['raised red or bluelesion'],
 'Acquired Immuno Deficiency Syndrome': ['early'],
 'Acute encephalitis syndrome': ['headache',
  'fever',
  'confusion',
  'stiff neck',
  'vomit'],
 'Alcohol Abuse and Alcoholism': ['drinking large amounts of alcohol over a long period',
  'difficulty cutting down',
  'acquiring and drinking alcohol taking up a lot of time',
  'usage resulting in problems',
  'withdrawaloccurring when stopping'],
 'Alopecia (hair loss)': ['loss ofhairfrom part of the head or body.'],
 'Alzheimer': ['memory loss',
  'problems with language',
  'disorientation',
  'mood swings'],
 'Amaurosis Fugax': ['temporary fleeting of vision in one or both eyes'],
 'Amblyopia': ['decreased vision'],
 'Amoebiasis': ['bloody diarrhea', 'abdominal pain'],
 'Anaemia': ['feeling tired',
  'pale skin',
  'weakness',
  'shortness of breath',
  'feeling faint'],
 'Aniseikonia': ['objects different sizes in 